# Import thư viện

In [1]:
import random

# Message và cover

In [2]:
with open('msg.txt', 'r', encoding='utf8') as f:
  msg = f.read()

In [3]:
print(msg)

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?


In [4]:
with open('cover.txt', 'r', encoding='utf-8') as f:
    cover = f.read()

In [5]:
print(cover)

xin chao cac ban



# Map table

In [6]:
invisible = {
    "\u200B",
    "\u200E",
    "\u202C",
    "\u202A",
    "\u202B",
    "\u202C",
    "\u202D",
}

bits_to_zwc = {
    "0001": "\u200B\u202A",
    "0010": "\u202A\u200B",
    "0011": "\u202C\u202A",
    "0100": "\u202A\u202C",
    "0101": "\u202C\u202D",
    "0110": "\u200B\u202C",
    "0111": "\u202C\u200B",
    "1000": "\u202A\u202B",
    "1001": "\u202B\u202A",
    "1010": "\u200E\u202D",
    "1011": "\u202D\u200E",
    "1100": "\u200E\u202C",
    "1101": "\u202C\u200E",
    "1110": "\u202D\u202C",
    "1111": "\u202D\u200B",
    "0000": "\u200B\u202D"
}

zwc_to_bits = {bits_to_zwc[x]: x for x in bits_to_zwc.keys()}

# Helper function

In [7]:

# Biến msg thành chuỗi bits 0 1
def to_bits(s: str) -> str:
    result: str = ""
    for c in s:
        bits = bin(ord(c))[2:]
        bits = bits.zfill(8)
        result += bits
    return result

# Biến chuỗi bit 0 1 thành msg
def from_bits(bits: str) -> str:
    chars: str = ""
    for b in range(len(bits) // 8):
        byte = bits[b * 8:(b + 1) * 8]
        chars += chr(int(''.join([bit for bit in byte]), 2))
    return chars


# Invert chuỗi bit 0 1
def invert(s: str) -> str:
    result = ""
    for c in s:
        if c == "0":
            result += "1"
        else:
            result += "0"
    return result

# Tách chuỗi bit thành các nhóm 4 bit
def separate(bits: str) -> list[str]:
    result: list[str] = []
    for b in range(len(bits) // 4):
        result.append(bits[b * 4: (b + 1) * 4])
    return result



# đối với từng cặp nhóm 4 bits, thực hiện phép xor dùng hàm xor_pair
def xor_separated_bits(separated_bits: list[str]) -> list[str]:

    # thực hiện xor trên 2 nhóm bits
    def xor_pair(first: str, second: str) -> str:
        assert len(first) == len(second)
        return bin(int(first, 2) ^ int(second, 2))[2:].zfill(4)

    result: list[str] = []
    for b in range(0, len(separated_bits), 2):
        result.append(xor_pair(separated_bits[b], separated_bits[b + 1]))
    return result

# đối với từng cặp nhóm 4 bits, merge bộ 4 bits đầu tiên và bộ 4 bits có được sau khi 
# đã thực hiện xor trên cặp này trước đó
def merged_xor(separated_bits: list[str], xor_ed_bits: list[str]) -> list[str]:
    result: list[str] = []
    for b in range(0, len(separated_bits), 2):
        result.append(separated_bits[b] + xor_ed_bits[b // 2])
    return result



# với từng chuỗi 8 bits sau khi merge_xor, đưa về ký tự vô hình qua hàm map_bits_to_zwc
def encode_to_zwc(merged_xor_ed: list[str]) -> str:

    # map chuỗi 4 bits sang ký tự vô hình (2 ký tự)
    def map_bits_to_zwc(bits: str, mapped: dict[str, str]) -> str:
        assert len(bits) == 4
        return mapped[bits]

    result = ""
    for bits in merged_xor_ed:
        result += map_bits_to_zwc(bits[:4], bits_to_zwc)
        result += map_bits_to_zwc(bits[4:], bits_to_zwc)
    return result

# extract ký tự vô hình từ stego text và convert từng cặp ký tự về chuỗi 4 bits
def extract_bits_from_stego(stego_text: str, mapped: dict[str, str]) -> str:
    hidden_char = ""
    for c in stego_text:
        if c in invisible:
            hidden_char += c
    bits = ""
    for c_index in range(0, len(hidden_char), 2):
        bits += mapped[hidden_char[c_index] + hidden_char[c_index + 1]]
    return bits


# Encryption

In [8]:
def key_gen(msg: str, key_file: str) -> str:
    msg_len = len(to_bits(msg))
    key = ''.join([str(random.randint(0, 1)) for i in range(msg_len)])
    with open(key_file, 'w') as f:
        f.write(key)
    return key


def xor_str(str1: str, str2: str) -> str:
    assert len(str1) == len(str2)
    result = ""
    for i in range(len(str1)):
        assert str1[i] == '0' or str1[i] == '1'
        assert str2[i] == '0' or str2[i] == '1'
        xor_ed = int(str1[i]) ^ int(str2[i])
        result += str(xor_ed)
    return result


def encrypt(message: str, key: str) -> str:
    assert len(message) == len(key)
    return xor_str(message, key)


def decrypt(encrypted: str, key: str) -> str:
    assert len(encrypted) == len(key)
    return xor_str(encrypted, key)


# Embed

In [9]:
def embed(msg_file: str, cover_file: str, stego_file: str = None, key=None):
    assert msg_file.endswith('.txt') and cover_file.endswith('.txt')
    if stego_file:
        assert stego_file.endswith('txt')
    with open(msg_file, 'r', encoding='utf-8') as f:
        msg = f.read()

    with open(cover_file, 'r', encoding='utf-8') as f:
        cover = f.read()

    msg_bits = to_bits(msg)

    if key:
      msg_bits = encrypt(msg_bits, key)

    inverted = invert(msg_bits)
    separated = separate(inverted)
    xor_ed = xor_separated_bits(separated)
    merged_xor_ed = merged_xor(separated, xor_ed)
    encoded = encode_to_zwc(merged_xor_ed)
    stego_text = encoded + cover

    out_file = stego_file
    if not out_file:
        file = cover_file.split('.')[0]
        out_file = file + '_embedded.txt'
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(stego_text)

# Extract

In [10]:
def extract(embedded_file: str, output_msg_file: str = None, key=None):
    with open(embedded_file, 'r', encoding='utf-8') as f:
        stego_text = f.read()
    bits = extract_bits_from_stego(stego_text, zwc_to_bits)
    bits_separated = separate(bits)
    xor_ed = xor_separated_bits(bits_separated)
    merged_xor_ed = merged_xor(bits_separated, xor_ed)

    msg_bits = invert("".join(merged_xor_ed))
    if key:
      msg_bits = decrypt(msg_bits, key)
    
    decoded = from_bits(msg_bits)

    out_file = output_msg_file
    if not out_file:
        file = embedded_file.split('.')[0]
        out_file = file + '_extracted.txt'
    with open(out_file, 'w', encoding='utf-8') as f:
        f.write(decoded)


# Key gen

In [11]:
key = key_gen(msg, 'key')
print(key)

1111111001010011110100110101001010000101110000101001011001001101101100000001100111011101000010000101001000000010111001010111000101001111110001000001101111000110011001010101101001100101011110101000010011111110010101011011000100011010111111011111100101101100000110010110001111101010100000011111100001010001011011111110010011011101010000100010010101111011101101110000110001010110010111101011111000011110101111001110001100111110110000000110010000001010110010010101000001010111110011001110000000110000010000000011100100111000000011111100111101110110110011001011000111110111000000111110001000111100101001011000110100111011011111101100110011011010101001011111111111011100111001111110000011000111111111001010010100000011100100100001001101110110010110001001111110001101100000111000011001110100100110101001111100101110001001001101000010100100011101111010011100000011001100111110111101100011010110111010101111100001000000100010101101001000001111101001110000101011010101100111101000001101010111011101101011111111

In [12]:
embed('msg.txt', 'cover.txt', 'embedded.txt', key)

In [13]:
with open('embedded.txt', 'r', encoding='utf8') as f:
  embeded = f.read()

In [14]:
print(embeded)

‬‭‬​‎‬‬‭‪‬‎‬‪‫‬‭​‭‭​‪‬‬‎‪‬‬‎‎‬‭‬‪​‪‫‫‪‬‎‬‭‪‬‪‫‭​‎‬‪‫‫‪‬​‬​‪‬‭‬​‪‎‬‪‫‬‭‬​‫‪‭‬​‪‪‫‭‬​‪‎‬‬​‭​​‪‭‬‭‬‬‭‭‬​‬‪‫‎‬‭‎‪​‪​‪‫‪‬‬​​‬‪​‪‬‭​‬‭‫‪‎‬‭‬​‬‬​‬​‬‭‭‎​‬‭​‎‬‬‪‭‬‎‭​‬‪‫‬‭‪‬‫‪‪‬‭‎‪‬‭​‫‪‬‪‫‪‫‪‬‭‬‎​‬‪‫‫‪‬‪​‪‪‫​‭‬‪‪‬‬‪‭​‭‎‎‬‬‭‬‭‭​‪‫‪‫‪‫‪‬‪​‬‎​‬‎‬‬‭‪‬‬‪‬​‬‎‎‭‪‫‫‪​‬‎‭‬‎‎‭‭‬‫‪‎‭‪‬​‪‭​‬‭‬‭‬​‪​‪​‬​‭‎‫‪‎‬​‬‭‬‎‭‪‬‬​‬‎​‪‬​‎‭​‪‭‬‬‪‬‭‫‪‬‭‪​‬‪‎‬​‬‭‎‬‭‬​​‬‭‎‬​‬‎​‪‫‪​‬‫‪‬‪‪‫‪‫​‪​‭‫‪‪‫‬‭‭‬‫‪‬‎‭‬​‭‫‪​‭‬​​‪​‭‬‭​‭‎‭​‪​‪​‭​‭‭​‎‭‭​‭‎​‪‪‬​‬‬​‎‬‭​‬‭‬‪‭‬‫‪‪‫‭‬‪​‬​​‬‭‬‪​‎‬‬‎‪​‪‬‬​​‭‫‪‬‭‭‎‪​‫‪‪​‭‬‭​​‭​‬‭‎‭‬‬‎‬‭‭​‭​‫‪‭‬‪‫‎‭‪‬‪‫‬​‬​‪‬​‭‫‪​‬​‪​‪‬‎​‪​‪‪​‬‭​‬‪‬‭‬‭​‬‎‪​‎‭‎‬‪‫‪‬‬‪​‬​‪‭‬‪​‪‬‎‭‪‬‪​​‪‎‭​‪‭​‬‎‎‬​‬‫‪‫‪‭‎‬‎‪‬‪‫‬​‭‬‭‎‬‎‪‬‎‬​‪​‭‬‭​‭​‭‫‪‬‎‭‎‎‭‬‪‎‬‪‬​‪‭​‎‬​‭‪‬‬‪‭‬‪​‎‭‪​‭‎‎‬‪‬‪​‬​​‪​‪‬‎‫‪‭‬‬‪​‭‬‎‬​​‭‬‭‬‭‭​​‭​‪‬​​‭​‬‭‎​‪‭‬‭​‎‭‬‪‬​‬‎‭‎‬‪​‪‪‫‭​‎‭​‬‪‬​‬‪​‬‎‪‬‬‪‪​​‬‬​‬‎‪​‪‬​‭​‬‪‫‬​‬‎​‭‭‬‪‬​‪‭‬‪‬‪​‬‭‭​‎‬‬‎​‬‭‬‬‪‪‬​‬‪​‫‪‬‪‪‫‬‪‬‎‬‪‬​‭‎‬‎‬​‪‬‭‬​‬‭‬‎‬‫‪‎‬​‬‭​‬‎‪‬‬‎‎‭‬‎‎‭​‪‪​​‪‫‪‭‬‭‎‪​​‬‬‭‫‪​‭‬‪‬‎‭‎‎‭‭‎‬‭‪‫‬​‭‬‪‫‬‭​‪‭​​‪‭‎‪‬‪‫‪‫‬‎‬‭‬‎‭​‭‬‬‎‭‎​‬‭‎‬‪​‭‫‪‬‭‭‬‬‭​‬‪‬‎‭​‬​‪‭‬‪‫‭‎‪‫​‬‪​​‬‬‪‬‭‭‎‭‎‭​​‭‬‎​‬‬‭‬‪‪‬‫‪‪‫‎‭‭‎​‭‪‫‎‭‪​​‭‭‎‫‪‬‎‭​‎‬‎‭

In [15]:
extract('embedded.txt', "extracted.txt", key)

In [16]:
with open('extracted.txt', 'r', encoding='utf8') as f:
  extracted = f.read()

In [17]:
print(extracted)

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?


In [18]:
print(msg)

Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?


In [19]:
assert extracted == msg